###Background

Safety problem is one of the most concerned issues when foreign university students choose their places of residence. So it's an analysis of crimes for foreign students studying at universities of San Francisco. And in the last, some suggestions on where to live and some advices on how to live will be given for students.

###Data Analysis

In [3]:
#packages and environment
from csv import reader
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import os

os.environ["PYSPARK_PYTHON"] = "python3"

In [4]:
#Download crime data of 2003-2018 from DataSF
import urllib.request

urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "/tmp/sf_03_18.csv")
dbutils.fs.mv("file:/tmp/sf_03_18.csv", "dbfs:/project1/sf_03_18.csv")
display(dbutils.fs.ls("dbfs:/project1"))


In [5]:
#Load data from Database
crimeDataPath = "dbfs:/project1/sf_03_18.csv"
df = spark.read.csv(crimeDataPath, header = "true")
display(df.take(10))

IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId
110308742,MISSING PERSON,FOUND PERSON,Friday,04/15/2011,16:00,TARAVAL,LOCATED,0 Block of CRESPI DR,-122.476327815126,37.7196874952717,POINT (-122.476327815126 37.7196874952717),11030874275000
090316914,WARRANTS,WARRANT ARREST,Thursday,03/26/2009,12:28,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411339562557,37.7812707434494,POINT (-122.411339562557 37.7812707434494),9031691463010
130132311,SECONDARY CODES,DOMESTIC VIOLENCE,Friday,02/15/2013,02:00,PARK,"ARREST, BOOKED",CASTRO ST / MARKET ST,-122.435187699349,37.7626702770872,POINT (-122.435187699349 37.7626702770872),13013231115200
080052833,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Tuesday,01/15/2008,20:13,MISSION,"ARREST, CITED",VALENCIA ST / 16TH ST,-122.421886357981,37.7649178909424,POINT (-122.421886357981 37.7649178909424),8005283365016
070535770,SECONDARY CODES,DOMESTIC VIOLENCE,Saturday,05/26/2007,18:10,MISSION,"ARREST, BOOKED",2800 Block of FOLSOM ST,-122.413935404044,37.7516932889916,POINT (-122.413935404044 37.7516932889916),7053577015200
130407330,OTHER OFFENSES,TRAFFIC VIOLATION,Friday,05/17/2013,21:56,SOUTHERN,"ARREST, CITED",500 Block of STEVENSON ST,-122.410793915482,37.780832063096,POINT (-122.410793915482 37.780832063096),13040733065015
136080803,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,04/07/2013,12:30,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.403404791479 37.775420706711),13608080306244
071177357,FORGERY/COUNTERFEITING,"FORGERY, DRIVERS LICENSE OR ID-CARD",Friday,11/16/2007,17:00,NORTHERN,PROSECUTED BY OUTSIDE AGENCY,2500 Block of VANNESS AV,-122.423907410647,37.7990497607213,POINT (-122.423907410647 37.7990497607213),7117735709120
060606109,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Wednesday,06/07/2006,19:45,NORTHERN,NONE,BUSH ST / BUCHANAN ST,-122.430170528945,37.7874391353148,POINT (-122.430170528945 37.7874391353148),6060610906243
050719825,ASSAULT,BATTERY OF A POLICE OFFICER,Tuesday,06/28/2005,01:02,BAYVIEW,"ARREST, BOOKED",NAPOLEON ST / EVANS AV,-122.396010154392,37.7472622922111,POINT (-122.396010154392 37.7472622922111),5071982504154


In [6]:
#data preprocessing
#convert the original date fromat from string to 'date' type

from pyspark.sql.functions import *
df = df.withColumn("Date", to_date(col("Date"), "MM/dd/yyyy"))\
       .withColumn('hour', hour(df['Time']))\
       .withColumn('year', year('Date'))\
       .withColumn('X', df['X'].cast('double'))\
       .withColumn('Y', df['Y'].cast('double'))

#cache data
df.cache()

Out[5]: DataFrame[IncidntNum: string, Category: string, Descript: string, DayOfWeek: string, Date: date, Time: string, PdDistrict: string, Resolution: string, Address: string, X: double, Y: double, Location: string, PdId: string, hour: int, year: int]

In [7]:
display(df.take(10))

IncidntNum,Category,Descript,DayOfWeek,Date,Time,PdDistrict,Resolution,Address,X,Y,Location,PdId,hour,year
110308742,MISSING PERSON,FOUND PERSON,Friday,2011-04-15,16:00,TARAVAL,LOCATED,0 Block of CRESPI DR,-122.476327815126,37.7196874952717,POINT (-122.476327815126 37.7196874952717),11030874275000,16,2011
090316914,WARRANTS,WARRANT ARREST,Thursday,2009-03-26,12:28,SOUTHERN,"ARREST, BOOKED",1000 Block of MARKET ST,-122.411339562557,37.7812707434494,POINT (-122.411339562557 37.7812707434494),9031691463010,12,2009
130132311,SECONDARY CODES,DOMESTIC VIOLENCE,Friday,2013-02-15,02:00,PARK,"ARREST, BOOKED",CASTRO ST / MARKET ST,-122.435187699349,37.7626702770872,POINT (-122.435187699349 37.7626702770872),13013231115200,2,2013
080052833,OTHER OFFENSES,"DRIVERS LICENSE, SUSPENDED OR REVOKED",Tuesday,2008-01-15,20:13,MISSION,"ARREST, CITED",VALENCIA ST / 16TH ST,-122.421886357981,37.7649178909424,POINT (-122.421886357981 37.7649178909424),8005283365016,20,2008
070535770,SECONDARY CODES,DOMESTIC VIOLENCE,Saturday,2007-05-26,18:10,MISSION,"ARREST, BOOKED",2800 Block of FOLSOM ST,-122.413935404044,37.7516932889916,POINT (-122.413935404044 37.7516932889916),7053577015200,18,2007
130407330,OTHER OFFENSES,TRAFFIC VIOLATION,Friday,2013-05-17,21:56,SOUTHERN,"ARREST, CITED",500 Block of STEVENSON ST,-122.410793915482,37.780832063096,POINT (-122.410793915482 37.780832063096),13040733065015,21,2013
136080803,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Sunday,2013-04-07,12:30,SOUTHERN,NONE,800 Block of BRYANT ST,-122.403404791479,37.775420706711,POINT (-122.403404791479 37.775420706711),13608080306244,12,2013
071177357,FORGERY/COUNTERFEITING,"FORGERY, DRIVERS LICENSE OR ID-CARD",Friday,2007-11-16,17:00,NORTHERN,PROSECUTED BY OUTSIDE AGENCY,2500 Block of VANNESS AV,-122.423907410647,37.7990497607213,POINT (-122.423907410647 37.7990497607213),7117735709120,17,2007
060606109,LARCENY/THEFT,PETTY THEFT FROM LOCKED AUTO,Wednesday,2006-06-07,19:45,NORTHERN,NONE,BUSH ST / BUCHANAN ST,-122.430170528945,37.7874391353148,POINT (-122.430170528945 37.7874391353148),6060610906243,19,2006
050719825,ASSAULT,BATTERY OF A POLICE OFFICER,Tuesday,2005-06-28,01:02,BAYVIEW,"ARREST, BOOKED",NAPOLEON ST / EVANS AV,-122.396010154392,37.7472622922111,POINT (-122.396010154392 37.7472622922111),5071982504154,1,2005


####The number of crimes of different types

In [9]:
crimeByCategory = df.select('Category').groupBy('Category').count().withColumnRenamed('count', 'num').orderBy('num',ascending = False).toPandas()

fig, axes = plt.subplots(1,1,figsize=(15,10))

sb.barplot('num','Category', data=crimeByCategory)
plt.grid(axis='x',linestyle='--')
plt.xlabel('Number of crimes')
plt.ylabel('Category of crimes')
plt.title('Number of crimes from 2003-2018 in SF')
plt.tight_layout()
display(fig)

####The number of crimes at differen district

In [11]:
#Count the number of crime at different district
crimeByDistrict = df.select('PdDistrict').groupBy('PdDistrict').count().withColumnRenamed('count','num').orderBy('num', ascending=False).toPandas()

#drop the None row of data
crimeByDistrict = crimeByDistrict.dropna(axis=0)

#draw the result
fig, axes = plt.subplots(1,1,figsize=(13,7))
plt.bar('PdDistrict','num',data = crimeByDistrict )
plt.ylabel('Number of crimes')
plt.title('Number of crimes at different district')
plt.xticks(rotation=-45)
plt.tight_layout()

display(fig)

According to the table, crimes occurs most frequently at Southern district. Then Mission district and Nothern district come at second and third. The security condition is best at Richmond.

Let's have a look at the trend of the number of crimes happend these years for these districts.

In [14]:
crimeDY = df.select('PdDistrict','year').groupBy('year','PdDistrict').count().orderBy('year').toPandas()

In [15]:
from pandas import Series, DataFrame
import pandas as pd
crimeByD = DataFrame(crimeDY[crimeDY['PdDistrict']=='TARAVAL'])
crimeByD = crimeByD.drop('PdDistrict',axis=1)
crimeByD.rename(columns={'count':'TARAVAL'}, inplace=True)

crimeByD['RICHMOND'] =crimeDY[crimeDY['PdDistrict']=='RICHMOND']['count'].tolist()

crimeByD['PARK'] =crimeDY[crimeDY['PdDistrict']=='PARK']['count'].tolist()
crimeByD['NORTHERN'] =crimeDY[crimeDY['PdDistrict']=='NORTHERN']['count'].tolist()
crimeByD['MISSION'] =crimeDY[crimeDY['PdDistrict']=='MISSION']['count'].tolist()
crimeByD['INGLESIDE'] =crimeDY[crimeDY['PdDistrict']=='INGLESIDE']['count'].tolist()
crimeByD['CENTRAL'] =crimeDY[crimeDY['PdDistrict']=='CENTRAL']['count'].tolist()
crimeByD['SOUTHERN'] =crimeDY[crimeDY['PdDistrict']=='SOUTHERN']['count'].tolist()
crimeByD['BAYVIEW'] =crimeDY[crimeDY['PdDistrict']=='BAYVIEW']['count'].tolist()
crimeByD['TENDERLOIN'] =crimeDY[crimeDY['PdDistrict']=='TENDERLOIN']['count'].tolist()

crimeByD = crimeByD.set_index('year')
crimeByD = crimeByD.drop(2018)

fig=crimeByD.plot(figsize=(15,7))
plt.xlabel('2003-2018')
plt.ylabel('Number of crimes')
display(fig)

According to the number of crimes trending of these districs, we can see that the Top3 safest distrcts are 'RICHIMOND', 'PARK' and 'TRAVEL', however, the number of yearly crimes keep going down at PARK district. We can infer that the security is getting better at this district.

####the number of crimes each "Sunday" at "SF downtown"

In [18]:
#define the area of "SF downtown"
#Let's make it a rectangle area which is ( ( -122.5038, 37.7080), (-122.3568, 37.7080), (-122.5038,37.8057), (-122.3568, 37.8057)
#So we can filter the crime happen at SF downton now

from pyspark.sql.functions import year
import matplotlib.dates as mdates
monthsFmt = mdates.DateFormatter('%m')

crimeAtDowntown = df.filter(df['x']> -122.5038)\
                    .filter(df['x']< -122.3568)\
                    .filter(df['y']< 37.8057)\
                    .filter(df['y']> 37.7080)\
                    .filter(df['DayOfWeek'] == 'Sunday')\
                    .select('Date')\
                    .groupBy('Date')\
                    .count()\
                    .orderBy(df['date'][6:10],df['date'][0:2],df['date'][3:5])
crimeAtDowntown_df = crimeAtDowntown.toPandas()

####Analysis the number of crime in each month of 2015, 2016, 2017

In [20]:
#filter data among 2015-2018
crimeByYear = df.filter(year(col('Date'))>2014)\
                .filter(year(col('Date'))<2018)
#convert to pandas dataframe, and group by year and month
crimeByYear_pd = crimeByYear.groupBy(year(col('Date')).alias('year'), month(col('Date')).alias('month')).count().orderBy('year','month').toPandas()

#group data by year and month
crimeByYear_pd = crimeByYear_pd.pivot(index='month', columns='year', values='count')
fig = crimeByYear_pd.plot(kind='line')
fig.set_title('Number of Crimes during 2015-2017')
display(fig)

according to the result shown above, the number of crimes keeps in the years of 15,16.

The number of crimes drops in December in 2017

####Analysis the number of crime with respect to the hour in certain day 

like 2015/12/15, 2016/12/15,/ 2017/12/15

In [23]:
#extract the data for the three days
crimeDay15 = df.filter(df['Date']=='2015-12-15').select('hour').groupBy('hour').count().orderBy('hour').toPandas()
crimeDay16 = df.filter(df['Date']=='2016-12-15').select('hour').groupBy('hour').count().orderBy('hour').toPandas()
crimeDay17 = df.filter(df['Date']=='2017-12-15').select('hour').groupBy('hour').count().orderBy('hour').toPandas()

In [24]:
fig,axes = plt.subplots(3,1,figsize=(10,7))

axes[0].plot(crimeDay15['hour'],crimeDay15['count'])
axes[1].plot(crimeDay16['hour'],crimeDay16['count'])
axes[2].plot(crimeDay17['hour'],crimeDay17['count'])

display(fig)

In [25]:
#put them together to compare three year on the same day

crimeDay15.rename(columns={'count':'2015'}).set_index('hour')
crimeDay16.rename(columns={'count':'2016'}).set_index('hour')
crimeDay17.rename(columns={'count':'2017'}).set_index('hour')

#when join together, different years have different columns of data
crimeDay = pd.concat([crimeDay15, crimeDay16, crimeDay17],axis=1).set_index('hour') 

#draw the fig
fig = crimeDay.plot(kind='line', figsize=(15,5))
fig.set_title('Number of Crimes in Different years on 12/15')
display(fig)

####Advice to distribute the police based on result
1) The top3 danger district

2) The crime event with respect to category and time(hour) from the result of step1

In [27]:
#get the total number of crimes in different districs
crimeByDistrict = df.select('PdDistrict').groupBy('PdDistrict').count().orderBy('count',ascending=False).toPandas()

fig, axes = plt.subplots(1,1, figsize=(10,7))
sb.barplot(x='PdDistrict', y='count', data = crimeByDistrict)

plt.ylabel('Number Of Crimes in Total')
plt.xticks(rotation=-45)
plt.title('Number of Crimes in Different Districs')
plt.tight_layout()

display(fig)

so the Top3 dangerous districs are 'Southern' 'Mission' 'Northern'

For different hours, the distribute of the crimes

In [30]:
#count the number of crime by hour in different district
crimeByHD = df.select('Category', 'hour').groupBy('Category', 'hour').count().orderBy('hour').toPandas()

#set index
crimeByHD = crimeByHD.pivot(index='Category', columns='hour', values='count')
crimeByHD = crimeByHD.div(crimeByHD.sum(axis=0),axis=1)
# display(crimeByHD)
#draw graph by hours and category
fig, axes = plt.subplots(4, 6, figsize=(30,18))

for i in crimeByHD.columns:
  labels = crimeByHD[i].sort_values(ascending=False)[:5].index.tolist()
  counts = crimeByHD[i].sort_values(ascending=False)[:5].values
  labels.append('other')
  counts = np.append(counts, 1-counts.sum())
  axes[i//6][i%6].pie(x=counts, labels=labels)
  axes[i//6][i%6].axis('equal')
  axes[i//6][i%6].set_title('Hour {}'.format(i))
  
fig.suptitle('Percent of Crimes for Different District All Day')
display(fig)
  
  

According to the data above, we can see that Larceny and theft are the main crimes and happens more during dinner period

####For different categroy of crime, find the percentage of resolution.

In [33]:
#create a table with the total number of crimes and the number of crimes resolved


crimes = df.groupBy('Category').count().withColumnRenamed('count','numTotal').orderBy('numTotal', ascending = False)

#Table with the number of crimes resolved
crimeResolved = df.filter(df['Resolution']!='NONE').groupBy('Category').count().withColumnRenamed('count','numResolved').orderBy('numResolved',ascending=False)

#Join the table together so we can compute the percent of crimes resolved
crimes = crimes.join(crimeResolved, 'Category')
crimes = crimes.toPandas()

#calculate the resolve rate for each kind of crime 
crimes['resolveRate'] = crimes['numResolved']/crimes['numTotal']
crimes = crimes.sort_values('resolveRate', ascending = False)



In [34]:
fig, axes = plt.subplots(1,1,figsize=(13,8))

sb.barplot(x='resolveRate',y='Category', data=crimes)
plt.xlabel('Resolve rate')
plt.ylabel('Categories of crimes')
plt.title('The rank of crime resolve rate for different categories of crime')
plt.grid(axis='x',linestyle='--')
plt.tight_layout()
display(fig)

we can see from the data above that the percentage of larceny or theft is the highest, but the resolve rate of it is the lowest, less than 10%. So residents should take necessary safety equipments like alarms to their properties safe, or get a property insurance.

####Apply Spark ML clustering for spatial data annlysis

In [37]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline

locationTrain = df.filter(year(col('date'))>='2013')\
                 .filter(year(col('date'))<='2017')
locationTrain = locationTrain.select('X','Y')\
                .groupBy('X','Y')\
                .count()\
                .orderBy('count', ascending=False)\
                .dropna()

# locationTest = locationTest.filter(year(col('Date'))=='2018')
# locationTest = locationTest.select('X','Y')\
#                .groupBy('X','Y')\
#                .count()\
#                .orderBy('count', ascending=False)\
#                .dropna()

vect = VectorAssembler(inputCols = ['X', 'Y'], outputCol = 'features')
kmeans = KMeans(k=5, seed=1)
pipeline = Pipeline(stages=[vect, kmeans])
KmModel = pipeline.fit(locationTrain)

locationRes = KmModel.transform(locationTrain)
locationRes = locationRes.toPandas()

fig, axes = plt.subplots(1,1,figsize=(15,15))
axes.scatter('X','Y',data=locationRes, c=(locationRes['prediction']), s=5)

display(fig)


###Conclusion & suggestions

1.Crimes in SF happens often. Larceny or theft comes first, so it's necessary to select a safe place to live. Assault comes fourth, so do not go out alone, especially during some dangerous hours.

2.The top3 safest districts are Richimon, Park and Travel. In thses districts, the number of yearly crimes keeps going down from 2013 in Park district. We can predict that the safety condition will go better in Park district. So Park district is reconmmended for students.

3.Sunday is a good time for students to hang out with their frineds. However, downtown is not a very good place to go. Many crimes happens at downtown on Sundays. If you want to go shop at downtown, go with your friends. 

4.In some of the months like March, May and August, crimes happens most, so don't hang around in thses months. You can go out in February, November, Decenber, because crimes happens less. However, the number of crimes seems keeps going less in 2017 since October, the security in SF seems getting better recent months.

5.Don't believe that the sun will drive crimes away. It works, but the safest time is during 1:00-7:00. However, maybe it's because no one likes hanging around at this time, so it's better to go out during 12:00-14:00, it looks safer. Do not go out when it gets dark.

6.Among thses crimes, larceny or theft comes highest, but the resolution rate comes lowest. Be careful of your property, including your vehicles, if get lost, don't expect to get them back. Make sure that you park them in safe parking lots, and it's better to have an alarm at home, it may works.